In [ ]:
import pandas as pd
import numpy as np
# from pytz import timezone
from datetime import datetime as dt
from typing import Optional
import pendulum

airline_id_mapping = pd.read_csv('data/airline_id.csv').set_index('two_letter')
# display(airline_id_mapping)
airport_timezone = pd.read_csv('data/airports.csv').set_index('code')['time_zone_id']
aircraft_id = pd.read_json('data/aircraft.json').drop_duplicates().set_index('icao')
# display(airport_timezone)
    

In [ ]:


file = './upload/fill_in_template.xlsx'
df = pd.read_excel(file, sheet_name='ToUpload', 
    dtype={
    # "Date": np.datetime64,
    "Flight number": str,
})

for column in ['Flight number', 'From', 'To', 'Aircraft', 'Registration', 'Seat number']:
    df[column] = df[column].apply(str).apply(str.upper)

for column in ['Flight class', 'Flight reason']:
    df.loc[(df[column].isna()) |(df[column].isna() == 0), column] = 1
display(df.head())


In [ ]:
# Fill in duration
def calc_duration(date: str, from_airport: str, dest_airport:str, dep_time: str, arr_time):
    from_tz = airport_timezone.get(from_airport)
    dest_tz = airport_timezone.get(dest_airport) 
    from_time = pendulum.parse(date.strftime('%Y-%m-%d')  + ' ' + str(dep_time), tz = from_tz)
    arr_time = pendulum.parse(date.strftime('%Y-%m-%d') + ' ' + str(arr_time), tz = dest_tz)
    time_diff = (arr_time - from_time).in_minutes()
    hours, minutes  = divmod(time_diff, 60)
    hours = hours if hours > 0 else hours + 24
    duration = f'{hours:02d}:{minutes:02d}:00'
    return duration

no_duration = df[df['Duration'].isnull()]
df['Duration'] = df.apply(lambda x: calc_duration(x.Date, x.From, x.To, x['Dep time'], x['Arr time']), axis=1)
# df

In [ ]:

df['Aircraft_id'] = df['Aircraft'].map(lambda x: aircraft_id['id'].get(x))
df['Aircraft_name'] = df['Aircraft'].map(lambda x: aircraft_id['label'].get(x))
# if na get by full name
no_aircraft = df['Aircraft_id'].isna()
df.loc[no_aircraft, 'Aircraft_id'] = df.loc[no_aircraft, 'Aircraft'].map(lambda x: aircraft_id.set_index('name')['id'].get(x))
df[no_aircraft]


In [ ]:
aircraft_id.set_index('name')

In [ ]:
# fill in airline id
# airline_codes = df['Flight number'].str.extract(r'(\D{2})')[0]
airline_codes = df['Flight number'].str.slice(0,2)

df['Airline_id'] = airline_codes.map(lambda x: airline_id_mapping.loc[x, 'Airline_id'])
df['Airline'] = airline_codes.map(lambda x: airline_id_mapping.loc[x, 'Airline'])
# df['Airline_id'] = airline_codes .map(lambda x: airline_id_mapping.get(x))

print(' airline id')
df[df['Airline_id'].isna()]

In [ ]:
display(df)
filename = f"upload/{dt.now().strftime('%Y-%m-%d-%H-%M')}.csv"
print(f'导出文件 {filename}')
# export_columns = [
#     'Date',
#     'Flight number',
#     'From', 
#     'To',
#     'Dep time',
    
# ]
# if input(f'Export to {filename}?') == 'y': 
df.to_csv(filename, index=False, encoding='utf-8-sig')

In [ ]:
df.columns